# Вначале сджойним таблички в одну 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv', parse_dates=['date'])
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
items.head()

In [ ]:
item_categories.head()

In [ ]:
sales_train.head()

In [ ]:
sales_train.dtypes

In [ ]:
sales_train.shape

In [ ]:
sales_train_with_cat_id = sales_train.join(items, on='item_id', 
                                           how='left', 
                                           lsuffix='',
                                           rsuffix='items').drop(columns = ['item_iditems'])
sales_train_with_cat_id_and_name = sales_train_with_cat_id.join(item_categories,
                                                                on='item_category_id', 
                                                                how='left',
                                                                lsuffix='',
                                                                rsuffix='cat').drop(columns = ['item_category_idcat'])
                                                               
sales_train = sales_train_with_cat_id_and_name
sales_train.head()

In [ ]:
sales_train.shape

In [ ]:
sales_train_with_cat_id_and_name.head()

Кроме того, для удобства отдельно запишем год и месяц

In [ ]:
sales_train['year'] = pd.DatetimeIndex(sales_train['date']).year
sales_train['month'] = pd.DatetimeIndex(sales_train['date']).month

In [ ]:
sales_train.head()

In [ ]:
sales_train.shape

In [ ]:
sales_train[['date', 'shop_id']].head()

In [ ]:
total_sales_per_day = sales_train[['date', 'item_cnt_day','item_price' ]].groupby(['date']).sum()
total_sales_per_day

In [ ]:
plt.figure(figsize=(30,8))
plt.subplot(2, 1, 1)

plt.plot(total_sales_per_day['item_cnt_day'], label='total_item_cnt')
plt.legend(loc='upper left')


plt.subplot(2, 1, 2)

plt.plot(total_sales_per_day['item_price'], label='total_item_price')
plt.legend(loc='upper left')



Большие пики попадают на декабрь 2013 и 2014 -- новогодние покупки, вполне себе. В остальном по графику можно видеть, что общего тренда на рост нет.

Посмотрим отдельно по годам

In [ ]:
sales_train_2013 = sales_train[sales_train['year'] == 2013]
sales_train_2014 = sales_train[sales_train['year'] == 2014]
sales_train_2015 = sales_train[sales_train['year'] == 2015]


Сугруппируем по месяцам

In [ ]:

sales_train_month_grouped_2013 = sales_train_2013[['month', 'item_cnt_day','item_price' ]].groupby(['month']).sum()
sales_train_month_grouped_2014 = sales_train_2014[['month', 'item_cnt_day','item_price' ]].groupby(['month']).sum()
sales_train_month_grouped_2015 = sales_train_2015[['month', 'item_cnt_day','item_price' ]].groupby(['month']).sum()



И соберем в общую табличку

In [ ]:
monthly_sales = sales_train_month_grouped_2013.join(sales_train_month_grouped_2014,lsuffix = 2013, rsuffix=2014)
monthly_sales['item_cnt_day2015'] = sales_train_month_grouped_2015['item_cnt_day']
monthly_sales['item_price2015'] = sales_train_month_grouped_2015['item_price']
monthly_sales

In [ ]:
plt.figure(figsize=(16,12))
plt.subplot(2, 1, 1)

plt.plot(monthly_sales['item_cnt_day2013'], label='total_item_cnt_day2013')
plt.plot(monthly_sales['item_cnt_day2014'], label='total_item_cnt_day2014')
plt.plot(monthly_sales['item_cnt_day2015'], label='total_item_cnt_day2015')

plt.legend(loc='upper left')


plt.subplot(2, 1, 2)

plt.plot(monthly_sales['item_price2013'], label='item_cnt_day2013')
plt.plot(monthly_sales['item_price2014'], label='item_cnt_day2014')
plt.plot(monthly_sales['item_price2015'], label='item_cnt_day2015')
plt.legend(loc='upper left')



Почему-то в 2015 году нет подъема в ноябре и декабре. С декабрем понятно -- данные рано закончились.

Графики странным образом упорядочены по-разному. Суммарная стоимость покупок уменьшалась два года подряд, но при этом в 2014 году количество покупок было больше, чем в 2013. Значит покупали более дешевые вещи? 

# Отдельно по магазинам

In [ ]:
sales_train_shop_grouped_2013 = sales_train_2013[['shop_id', 'item_cnt_day','item_price' ]].groupby(['shop_id']).sum()
sales_train_shop_grouped_2014 = sales_train_2014[['shop_id', 'item_cnt_day','item_price' ]].groupby(['shop_id']).sum()
sales_train_shop_grouped_2015 = sales_train_2015[['shop_id', 'item_cnt_day','item_price' ]].groupby(['shop_id']).sum()


shop_sales = sales_train_shop_grouped_2013.join(sales_train_shop_grouped_2014,lsuffix = 2013, rsuffix=2014)
shop_sales['item_cnt_day2015'] = sales_train_shop_grouped_2015['item_cnt_day']
shop_sales['item_price2015'] = sales_train_shop_grouped_2015['item_price']
shop_sales

In [ ]:
plt.figure(figsize=(25,12))

x = list(shop_sales.index.values)
def shift(x, w):
    return [item + w for item in x]

plt.subplot(2, 1, 1)

plt.bar(x, list(shop_sales['item_cnt_day2013']), width = 0.3, alpha = 0.5, label='cnt_2013')
plt.bar(shift(x, 0.3), list(shop_sales['item_cnt_day2014']), width = 0.3, alpha = 0.5, label='cnt_2014')
plt.bar(shift(x, 0.6), list(shop_sales['item_cnt_day2015']), width = 0.3, alpha = 0.5, label='cnt_2015')

plt.legend(loc='upper left')


plt.subplot(2, 1, 2)

plt.bar(x, list(shop_sales['item_price2013']), width = 0.3, alpha = 0.5, label='sales_2013')
plt.bar(shift(x, 0.3), list(shop_sales['item_price2014']), width = 0.3, alpha = 0.5, label='sales_2014')
plt.bar(shift(x, 0.6), list(shop_sales['item_price2015']), width = 0.3, alpha = 0.5, label='sales_2015')

plt.legend(loc='upper left')


Видно, что помагазинные сравнения годов 2013, 2014 и 2015 похожи на то, что происходит и в суммарном графике. Опять количество продаж и суммарная стоимоть не коррелирует

Еще можно заметить, что есть 3 - 5 магазинов, в которых объем сильно больше остальных. 

# По категориям 

In [ ]:
sales_train_cat_grouped_2013 = sales_train_2013[['item_category_id', 'item_cnt_day','item_price' ]].groupby(['item_category_id']).sum()
sales_train_cat_grouped_2014 = sales_train_2014[['item_category_id', 'item_cnt_day','item_price' ]].groupby(['item_category_id']).sum()
sales_train_cat_grouped_2015 = sales_train_2015[['item_category_id', 'item_cnt_day','item_price' ]].groupby(['item_category_id']).sum()


cat_sales = sales_train_cat_grouped_2013.join(sales_train_cat_grouped_2014,lsuffix = 2013, rsuffix=2014)
cat_sales['item_cnt_day2015'] = sales_train_cat_grouped_2015['item_cnt_day']
cat_sales['item_price2015'] = sales_train_cat_grouped_2015['item_price']
cat_sales

In [ ]:
plt.figure(figsize=(25,12))

x = list(cat_sales.index.values)
def shift(x, w):
    return [item + w for item in x]

plt.subplot(2, 1, 1)

plt.bar(x, list(cat_sales['item_cnt_day2013']), width = 0.3, alpha = 0.5, label='cat_cnt_2013')
plt.bar(shift(x, 0.3), list(cat_sales['item_cnt_day2014']), width = 0.3, alpha = 0.5, label='cat_cnt_2014')
plt.bar(shift(x, 0.6), list(cat_sales['item_cnt_day2015']), width = 0.3, alpha = 0.5, label='cat_cnt_2015')

plt.legend(loc='upper left')


plt.subplot(2, 1, 2)

plt.bar(x, list(cat_sales['item_price2013']), width = 0.3, alpha = 0.5, label='cat_sales_2013')
plt.bar(shift(x, 0.3), list(cat_sales['item_price2014']), width = 0.3, alpha = 0.5, label='cat_sales_2014')
plt.bar(shift(x, 0.6), list(cat_sales['item_price2015']), width = 0.3, alpha = 0.5, label='cat_sales_2015')

plt.legend(loc='upper left')


Опять же хорошо видно, что есть несколько выдающихся категорий. И на глаз тут уже лучше коррелирует все. 